## Correlation of scenarios

This notebook produces Tab. 3 of the Extended Data. It investigates how much the MEASURED rankings of different transfer scenarios correlate with each other.

In [1]:
import mml.interactive
from pathlib import Path
mml.interactive.init(Path('~/.config/mml.env').expanduser())
from mml_tf.experiments import load_experiment, EXPERIMENTS, METRICS
from mml_tf.aggregate import AggregateStrategy, aggregate_observations
from mml_tf.tasks import get_valid_sources, test_tasks
from itertools import product
import pandas as pd
import numpy as np
import scipy.stats

/home/scholzpa/Documents/development/gitlab/mml/src/mml/interactive/__init__.py:49: UserWarning: Did not provide a "env_path", neither found set "MML_ENV_PATH" variable, you might need to provide "env_path" to "init" in order to use "mml" interactively in a jupyter/ipython setting.
  warnings.warn(
/home/scholzpa/miniconda3/envs/mml/lib/python3.8/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


 _____ ______   _____ ______   ___
|\   _ \  _   \|\   _ \  _   \|\  \
\ \  \\\__\ \  \ \  \\\__\ \  \ \  \
 \ \  \\|__| \  \ \  \\|__| \  \ \  \
  \ \  \    \ \  \ \  \    \ \  \ \  \____
   \ \__\    \ \__\ \__\    \ \__\ \_______\
    \|__|     \|__|\|__|     \|__|\|_______|
         ____  _  _    __  _  _  ____  _  _
        (  _ \( \/ )  (  )( \/ )/ ___)( \/ )
         ) _ ( )  /    )( / \/ \\___ \ )  /
        (____/(__/    (__)\_)(_/(____/(__/
Interactive MML API initialized.


In [2]:
corr_df = pd.DataFrame(columns=EXPERIMENTS, index=EXPERIMENTS)
for exp_1, exp_2 in product(EXPERIMENTS, repeat=2):
    _sims = []
    for metric in METRICS:
        for agg in [AggregateStrategy.FIRST, AggregateStrategy.SECOND, AggregateStrategy.THIRD]:
            exp_1_df = aggregate_observations(load_experiment(experiment_name=exp_1, metric=metric), strat=agg)
            exp_2_df = aggregate_observations(load_experiment(experiment_name=exp_2, metric=metric), strat=agg)
            for target in test_tasks:
                sources = get_valid_sources(target)
                _sims.append(exp_1_df[target].loc[sources].corr(exp_2_df[target].loc[sources], method= lambda x, y: scipy.stats.weightedtau(x, y)[0]))
    corr_df.at[exp_1, exp_2] = f'{np.mean(_sims):.3f} ± {np.std(_sims):.3f}'
corr_df

,Model<br>Architecture,Pretraining<br>Data,Augmentation<br>Policy,Co-Training<br>Data
Model<br>Architecture,1.000 ± 0.000,0.084 ± 0.147,0.078 ± 0.141,0.052 ± 0.137
Pretraining<br>Data,0.084 ± 0.147,1.000 ± 0.000,0.037 ± 0.164,0.038 ± 0.136
Augmentation<br>Policy,0.078 ± 0.141,0.037 ± 0.164,1.000 ± 0.000,-0.002 ± 0.140
Co-Training<br>Data,0.052 ± 0.137,0.038 ± 0.136,-0.002 ± 0.140,1.000 ± 0.000
